In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Итоговый алгоритм без LLM части. LLM прикручивается в Backend

![](https://sun9-60.userapi.com/impg/M_0H62JeBppFie1LfqVpMKO59k_p4Rjd7x5J6A/-1co6l8WoOc.jpg?size=1280x720&quality=96&sign=7742f82410293ae8912c48a5f93ab033&type=album)

In [2]:
!pip install catboost
!pip install natasha

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 70.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=5cf59ab47b012dd7f67095d6586cf3dd909d92fdaa88bd903e15b3a27ac09212
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26096 sha256=dc38c2b53450da2acf783fca993668b362601383912d3f5712cd0c7fd6d45258
  Stored in directory: /root/.cache/pip/wheels/fa/80/8c/43488a924a046b733b64de3fac99252674c

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
import catboost
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import nltk
from natasha import (Segmenter, MorphVocab, NewsEmbedding, NewsMorphTagger, NewsSyntaxParser, NewsNERTagger, PER, NamesExtractor, DatesExtractor, MoneyExtractor, AddrExtractor, Doc)
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib
from catboost import CatBoostClassifier

Класс для преобразования вопросов в вектора

In [4]:
import torch
from transformers import AutoModel, AutoTokenizer

from typing import List

class Embeddings:
    def __init__(self, model_name: str, device: torch.device) -> None:
        self.device = device
        self.model = AutoModel.from_pretrained(model_name).to(device)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

    def _average_pool(self, last_hidden_states: torch.tensor,
                 attention_mask: torch.tensor) -> torch.tensor:
        last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
        return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

    def exec(self, sentences: List[str]) -> torch.tensor:
        """Get embeddings for sentences

        Args:
            sentences (List[str]): list of sentences to embed

        Returns:
            torch.tensor: return tensor of embeddings
        """
        formatted_sentences = [f'query: {sentence}' for sentence in sentences]

        tokenized_sentences = self.tokenizer(
            formatted_sentences, max_length=512,
            padding=True, truncation=True, return_tensors='pt'
        )

        tokenized_sentences['input_ids'] = tokenized_sentences['input_ids'].to(self.device)
        tokenized_sentences['attention_mask'] = tokenized_sentences['attention_mask'].to(self.device)

        self.model.eval()
        outputs = []
        with torch.no_grad():
            for i in range(len(formatted_sentences)):
                batch = {'input_ids': tokenized_sentences['input_ids'][i].unsqueeze(0), 'attention_mask': tokenized_sentences['attention_mask'][i].unsqueeze(0)}
                out = self.model(**batch)
                embeddings = self._average_pool(out.last_hidden_state, tokenized_sentences['attention_mask'][i].unsqueeze(0))
                outputs.append(embeddings.squeeze(0))
        return torch.stack(outputs).cpu()

Загрузка данных

In [5]:
answer_class = pd.read_csv('/content/drive/MyDrive/хаки/ЦФО/answer_class.csv')

In [6]:
data = pd.read_csv('/content/drive/MyDrive/хаки/ЦФО/aug_train_data.csv')
data['Category']=data['Category'].astype('category')


In [7]:
import torch
device = torch.device('cuda:0')

In [8]:
embeddings = Embeddings('intfloat/multilingual-e5-large', device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [9]:
#считает эмбеддинги и записывает их в pd DataFrame
def get_embs(df):
  emb = embeddings.exec(df['Question'])
  X = pd.DataFrame(emb.numpy())
  return X

In [10]:

nltk.download('stopwords')
stop_words = set(stopwords.words("russian"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [11]:

segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

In [12]:
def lematizate_sentance(text):
  doc = Doc(text)
  doc.segment(segmenter)
  doc.tag_morph(morph_tagger)

  new_sent=""
  for token in doc.tokens:
      token.lemmatize(morph_vocab)
      if token.lemma not in stop_words:
        new_sent += f"{token.lemma} "
  return new_sent

def lematizate_sentances(texts):
  new_texts=[]
  for text in texts:
    new_texts.append(lematizate_sentance(text))

  return new_texts


In [13]:
#Технические заполнители, дальше будут удалены с помощью drop
def tech_fill(df):
  df['Category'] = [None]*len(df)
  df['answer_class'] = [None]*len(df)
  df['index'] = [None]*len(df)
  return df

In [14]:
def data_prep(texts, tfidf_vectorizer):
  #Перевод списка вопросов к нужному формату
  train_data = pd.DataFrame([])
  train_data['Question']=texts
  train_data=tech_fill(train_data)

  #добавление TF-IDF
  documents=lematizate_sentances(texts)
  tfidf_matrix = tfidf_vectorizer.transform(documents)
  tfidf_table = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
  train_data=pd.concat([train_data,tfidf_table], axis=1)

  return train_data

def stages_pred(X, model_cl_category, model_cl_answer, verbose=0):
  X_= get_embs(X)
  X = pd.concat([X,X_], axis = 1)
  X = X.drop(['Question','Category','answer_class','index'], axis= 1 )
  pred1=model_cl_category.predict(X)

  if verbose==1:
    print(f"Категория {pred1}")

  df=pd.DataFrame([])
  df['Category']=pred1[:,0]
  X = pd.concat([df, X], axis=1)
  X['Category']=X['Category'].astype('category')
  pred2=model_cl_answer.predict(X)
  return pred2

def pipeline_predict(text,model_cl_category, model_cl_answer, tfidf_vectorizer):
  #предсказание
  answer=stages_pred(data_prep(text, tfidf_vectorizer),model_cl_category, model_cl_answer, verbose=1)
  list_answers=[]
  for a in answer:

    list_answers.append(answer_class.iloc[a]['Answer'].values)


  return list_answers

In [15]:

model_cl_category = CatBoostClassifier()
model_cl_category.load_model('/content/drive/MyDrive/хаки/ЦФО/model_cl_category.bin')
model_cl_answer = CatBoostClassifier()
model_cl_answer.load_model('/content/drive/MyDrive/хаки/ЦФО/model_cl_answer.bin')

# Загрузка TfidfVectorizer из файла
tfidf_vectorizer = joblib.load("/content/drive/MyDrive/хаки/ЦФО/tfidf_vectorizer.pkl")

In [16]:
text=['когда я залутаю диплом?', 'во сколько встреча?']


In [17]:
answers=pipeline_predict(text,model_cl_category, model_cl_answer, tfidf_vectorizer)

Категория [['Документы']
 ['Организация уроков']]


In [18]:
answers[0]

array(['Можем его выдать, если вы:\n\nоплатили обучение после 3 декабря 2019 года;\nимеете среднее или высшее профессиональное образование — подойдут диплом СПО или ВО. Если вы ещё не закончили СУЗ или ВУЗ, подойдёт справка из образовательного учреждения;\nпри зачислении предоставили пакет документов — паспорт, диплом, СНИЛС, сведения о смене ФИО (при наличии), сведения о признании иностранного диплома (при наличии);\nсдали все промежуточные аттестации до даты итоговой аттестации;\nуспешно сдали итоговую аттестацию.\nИтоговой аттестацией может быть:\n\nитоговый экзамен;\nзащита проекта;\nдругие формы итоговой аттестационной работы.\nМы подготовим диплом в течение 30 дней от даты итоговой аттестации. Чтобы его получить, обратитесь к куратору.'],
      dtype=object)

-----------------------------

In [19]:
def f_data_prep2(texts):
  #Перевод списка вопросов к нужному формату
  train_data = pd.DataFrame([])
  train_data['Question']=texts

  return train_data

def f_pred2(texts, model):
   df = f_data_prep2(texts)
   emb = embeddings.exec(df['Question'])
   X = pd.DataFrame(emb.numpy())
   return model.predict_proba(X)


In [20]:
model_filter_model = CatBoostClassifier()
model_filter_model.load_model('/content/drive/MyDrive/хаки/ЦФО/filter_model.bin')

In [24]:
def filter_question(text,model_filter_model):
  pred = f_pred2(text, model_filter_model)
  tf_pred=pred>0.3
  return tf_pred


In [26]:
filter_question(['Когда появится рассписание?'],model_filter_model)

array([[ True,  True],
       [ True,  True]])